## 11.6 VeryNobleApp

W tej aplikacji wykorzystamy darmowe api ze strony [nabelprize.org](https://app.swaggerhub.com/apis/NobelMedia/NobelMasterData/2.1) do stworzenia aplikacji wyświetlającej nagrody nobla w architekturze `MVVM`. Aplikacja będzie zawierała cztery fragmenty:
- `NobelAwardsFragment` wyświetlający listę (`RecyclerView`) wszystkich nagród według kategorii
- `NobelAwardFragment` wyświetlający więcej informacji na temat wybranej na poprzednim ekranie nagrody, zawiera również listę (`RecyclerView`) wszystkich laureatów nagrody
- `LaureateFragment` wyświetlający więcej informacji na temat wybranego na poprzednim ekranie laureata, również zawiera `RecyclerView` ze wszystkimi nagrodami otrzymanymi przez danego laureata
- `WikiFragment` wyświetlający wpis na wikipedii dotyczący wybranego laureate - strona wyświetlana jest za pomocą `WebView`.

<table><tr><td><img src="https://media0.giphy.com/media/NuuG5HnAdmdTjLV0DV/giphy.webp" width="200" /></td><td><img src="https://media3.giphy.com/media/6pIR2WozCb81CcuGZo/giphy.webp" width="200" /></td><td><img src="https://media0.giphy.com/media/8QoE7dYN8as4gRjE16/giphy.webp" width="200" /></td></tr></table>


Tutaj mniejszy nacisk przyłożymy na warstwę danych (zostanie wygenerowana automatycznie) - skupimy się na implementacji architektury `MVVM` wykorzystując bibliotekę `Retrofit`. Wykorzystamy pojedyncze repozytorium oraz zaimplementujemy osobny `ViewModel` dla każdego fragmentu, który tego wymaga.

Rozpocznijmy od zmodyfikowania skryptów `gradle`, do pliku `gradle(Project)` dodajemy możliwość przesyłania argumentów przez `Navigation`

In [ ]:
buildscript { // przed blokiem plugins
    repositories {
        google()
    }
    dependencies {
        classpath "androidx.navigation:navigation-safe-args-gradle-plugin:2.5.2"
    }
}

Uzupełniamy również plik `gradle(Module)`

In [ ]:
plugins {
    id 'com.android.application'
    id 'androidx.navigation.safeargs'
}
...
android {
    ...
    buildFeatures {
        viewBinding true
    }
}
...
dependencies {
    // ViewModel
    implementation 'androidx.lifecycle:lifecycle-viewmodel:2.5.1'
    // LiveData
    implementation "androidx.lifecycle:lifecycle-livedata:2.5.1"

    // Navigation
    implementation "androidx.navigation:navigation-fragment:2.5.2"
    implementation "androidx.navigation:navigation-ui:2.5.2"

    // Retrofit
    implementation 'com.squareup.retrofit2:retrofit:2.9.0'
    implementation 'com.squareup.retrofit2:converter-gson:2.9.0'

    // OkHttp
    implementation 'com.squareup.okhttp3:logging-interceptor:4.10.0'
    ...
}

### **`Navigation`**

Nawigacja w tej aplikacji będzie liniowa - z jednego fragmetu można przejść do jednego innego fragmentu i z powrotem. Dodajmy nawigację i utwórzmy odpowiednie akcje

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<navigation xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/navigation"
    app:startDestination="@id/nobelAwardsFragment">

    <fragment
        android:id="@+id/nobelAwardsFragment"
        android:name="pl.udu.uwr.pum.verynobleappjava.ui.
fragments.nobleawards.NobelAwardsFragment"
        android:label="Nagrody Nobla"
        tools:layout="@layout/fragment_nobel_awards" >
        <action
            android:id="@+id/action_nobelAwardsFragment_to_nobelAwardFragment"
            app:destination="@id/nobelAwardFragment" >
        </action>
    </fragment>
    <fragment
        android:id="@+id/nobelAwardFragment"
        android:name="pl.udu.uwr.pum.verynobleappjava.ui.fragments.nobelaward.NobelAwardFragment"
        android:label="Nagroda Nobla"
        tools:layout="@layout/fragment_nobel_award" >
        <action
            android:id="@+id/action_nobelAwardFragment_to_nobelAwardsFragment"
            app:destination="@id/nobelAwardsFragment" />
        <action
            android:id="@+id/action_nobelAwardFragment_to_laureateFragment"
            app:destination="@id/laureateFragment" >
        </action>
    </fragment>
    <fragment
        android:id="@+id/laureateFragment"
        android:name="pl.udu.uwr.pum.verynobleappjava.ui.fragments.laureate.LaureateFragment"
        android:label="Laureat"
        tools:layout="@layout/fragment_laureate" >
        <action
            android:id="@+id/action_laureateFragment_to_nobelAwardFragment"
            app:destination="@id/nobelAwardFragment" />
        <action
            android:id="@+id/action_laureateFragment_to_wikiLaureateFragment"
            app:destination="@id/wikiLaureateFragment" >
        </action>
    </fragment>
    <fragment
        android:id="@+id/wikiLaureateFragment"
        android:name="pl.udu.uwr.pum.verynobleappjava.ui.fragments.wiki.WikiLaureateFragment"
        android:label="Wikipedia"
        tools:layout="@layout/fragment_wiki_laureate" />
</navigation>

### **`Retrofit`**

Wszystkie fragmenty wraz z aktywnością umieszczam w pakiecie `ui`. Dodajmy pakiet `api` do którego dodamy interfejs z metodami dostępowymi (których ciało zostanie wygenerowane przez `Retrofit`), oraz instancję samoego `Retrofit`. Wpierw przyjrzyjmy się naszemu [api](https://app.swaggerhub.com/apis/NobelMedia/NobelMasterData/2.1) - do uzyskania listy wszystkich nagród nobla o zadanej kategorii wykorzystamy endpoint **/nobelPrizes**, bazowym adresem `url` jest https://api.nobelprize.org/ - pełne zapytanie będzie wyglądać następująco
  'https://api.nobelprize.org/2.1/nobelPrizes?limit=30&sort=desc&nobelPrizeYear=2000&yearTo=2021&nobelPrizeCategory=eco&format=json'

Mamy tutaj serię parametrów:
```xml
- name: `offset`  
  in: query  
  description: Liczba elementów pominiętych przed rozpoczęciem zbierania  
  type: `integer`  
  minimum: 1  

- name: `limit`  
  in: query  
  description: Liczba zwróconych elementów  
  type: `integer`  
  minimum: 1

- name: `sort`  
  in: query  
  description: Kolejność sortowania  
  type: `string`  
  enum: ["asc", "desc"]  

- name: `nobelPrizeYear`  
  in: query  
  description: Rok przyznania nagrody nobla  
  type: `integer`  
  minimum: 1901

- name: `yearTo`  
  in: query  
  description: Z `nobelPrizeYear` wykorzystywany do wygenerowania zakresu  
  type: `integer`  
  minimum: 1901

- name: `nobelPrizeCategory`  
  in: query  
  description: Kategoria nagrody nobla  
  type: `string`  
  enum: ["che", "eco", "lit", "pea", "phy", "med"]

- name: `format`  
  in: query  
  description: Format wyjścia - domyślnie json  
  type: `string`  
  enum: ["json", "csv"]

- name: `csvLang`  
  in: query  
  description: Język wyjścia - domyślnie angielski  
  type: `string`  
  enum: ["en", "se", "no"]
```

Przykładowy wynik zapytania   https://api.nobelprize.org/2.1/nobelPrizes?limit=1&sort=desc&nobelPrizeYear=2000&nobelPrizeCategory=phy

```json
{
  "nobelPrizes": [
    {
      "awardYear": "2000",
      "category": {
        "en": "Physics",
        "no": "Fysikk",
        "se": "Fysik"
      },
      "categoryFullName": {
        "en": "The Nobel Prize in Physics",
        "no": "Nobelprisen i fysikk",
        "se": "Nobelpriset i fysik"
      },
      "dateAwarded": "2000-10-10",
      "topMotivation": {
        "en": "for basic work on information and communication technology"
      },
      "prizeAmount": 9000000,
      "prizeAmountAdjusted": 11538617,
      "links": [
        {
          "rel": "nobelPrize",
          "href": "https://api.nobelprize.org/2/nobelPrize/phy/2000",
          "action": "GET",
          "types": "application/json"
        }
      ],
      "laureates": [
        {
          "id": "726",
          "knownName": {
            "en": "Zhores Alferov"
          },
          "fullName": {
            "en": "Zhores I. Alferov"
          },
          "portion": "1/4",
          "sortOrder": "1",
          "motivation": {
            "en": "for developing semiconductor heterostructures used in high-speed- and opto-electronics",
            "se": "för utvecklingen av halvledarheterostrukturer för höghastighets- och optoelektronik"
          },
          "links": [
            {
              "rel": "laureate",
              "href": "https://api.nobelprize.org/2/laureate/726",
              "action": "GET",
              "types": "application/json"
            }
          ]
        },
        {
          "id": "727",
          "knownName": {
            "en": "Herbert Kroemer"
          },
          "fullName": {
            "en": "Herbert Kroemer"
          },
          "portion": "1/4",
          "sortOrder": "2",
          "motivation": {
            "en": "for developing semiconductor heterostructures used in high-speed- and opto-electronics",
            "se": "för utvecklingen av halvledarheterostrukturer för höghastighets- och optoelektronik"
          },
          "links": [
            {
              "rel": "laureate",
              "href": "https://api.nobelprize.org/2/laureate/727",
              "action": "GET",
              "types": "application/json"
            }
          ]
        },
        {
          "id": "728",
          "knownName": {
            "en": "Jack Kilby"
          },
          "fullName": {
            "en": "Jack S. Kilby"
          },
          "portion": "1/2",
          "sortOrder": "3",
          "motivation": {
            "en": "for his part in the invention of the integrated circuit",
            "se": "för hans del i uppfinningen av den integrerade kretsen"
          },
          "links": [
            {
              "rel": "laureate",
              "href": "https://api.nobelprize.org/2/laureate/728",
              "action": "GET",
              "types": "application/json"
            }
          ]
        }
      ]
    }
  ],
  "meta": {
    "offset": 0,
    "limit": 1,
    "nobelPrizeYear": 2000,
    "nobelPrizeCategory": "phy",
    "count": 1,
    "terms": "https://www.nobelprize.org/about/terms-of-use-for-api-nobelprize-org-and-data-nobelprize-org/",
    "license": "https://www.nobelprize.org/about/terms-of-use-for-api-nobelprize-org-and-data-nobelprize-org/#licence",
    "disclaimer": "https://www.nobelprize.org/about/terms-of-use-for-api-nobelprize-org-and-data-nobelprize-org/#disclaimer"
  }
}
```

Możemy wynik zmapować przy pomocy pluginu w Android Studio - `JSONToJava` - w efekcie dostaniemy klasę zawierającą całą serię klas wewnętrznych, które dodajemy do pakietu `data`

In [ ]:
public class NobelAwardsResponse implements Serializable {
  private Meta meta;

  private Links links;

  private List<NobelPrizes> nobelPrizes;

  public Meta getMeta() {
    return this.meta;
  }

  public void setMeta(Meta meta) {
    this.meta = meta;
  }

  public Links getLinks() {
    return this.links;
  }

  public void setLinks(Links links) {
    this.links = links;
  }

  public List<NobelPrizes> getNobelPrizes() {
    return this.nobelPrizes;
  }

  public void setNobelPrizes(List<NobelPrizes> nobelPrizes) {
    this.nobelPrizes = nobelPrizes;
  }

  public static class Meta implements Serializable {
    private String license;

    private Integer offset;

    private Integer nobelPrizeYear;

    private String terms;

    private Integer limit;

    private Integer count;

    private String disclaimer;

    private Integer yearTo;

    private String nobelPrizeCategory;

    public String getLicense() {
      return this.license;
    }

    public void setLicense(String license) {
      this.license = license;
    }

    public Integer getOffset() {
      return this.offset;
    }

    public void setOffset(Integer offset) {
      this.offset = offset;
    }

    public Integer getNobelPrizeYear() {
      return this.nobelPrizeYear;
    }

    public void setNobelPrizeYear(Integer nobelPrizeYear) {
      this.nobelPrizeYear = nobelPrizeYear;
    }

    public String getTerms() {
      return this.terms;
    }

    public void setTerms(String terms) {
      this.terms = terms;
    }

    public Integer getLimit() {
      return this.limit;
    }

    public void setLimit(Integer limit) {
      this.limit = limit;
    }

    public Integer getCount() {
      return this.count;
    }

    public void setCount(Integer count) {
      this.count = count;
    }

    public String getDisclaimer() {
      return this.disclaimer;
    }

    public void setDisclaimer(String disclaimer) {
      this.disclaimer = disclaimer;
    }

    public Integer getYearTo() {
      return this.yearTo;
    }

    public void setYearTo(Integer yearTo) {
      this.yearTo = yearTo;
    }

    public String getNobelPrizeCategory() {
      return this.nobelPrizeCategory;
    }

    public void setNobelPrizeCategory(String nobelPrizeCategory) {
      this.nobelPrizeCategory = nobelPrizeCategory;
    }
  }

  public static class Links implements Serializable {
    private String last;

    private String self;

    private String first;

    public String getLast() {
      return this.last;
    }

    public void setLast(String last) {
      this.last = last;
    }

    public String getSelf() {
      return this.self;
    }

    public void setSelf(String self) {
      this.self = self;
    }

    public String getFirst() {
      return this.first;
    }

    public void setFirst(String first) {
      this.first = first;
    }
  }

  public static class NobelPrizes implements Serializable {
    private String awardYear;

    private CategoryFullName categoryFullName;

    private Integer prizeAmount;

    private Integer prizeAmountAdjusted;

    private String dateAwarded;

    private List<? extends Links> links;

    private List<Laureates> laureates;

    private CategoryFullName category;

    private Laureates.Motivation topMotivation;

    public String getAwardYear() {
      return this.awardYear;
    }

    public void setAwardYear(String awardYear) {
      this.awardYear = awardYear;
    }

    public CategoryFullName getCategoryFullName() {
      return this.categoryFullName;
    }

    public void setCategoryFullName(CategoryFullName categoryFullName) {
      this.categoryFullName = categoryFullName;
    }

    public Integer getPrizeAmount() {
      return this.prizeAmount;
    }

    public void setPrizeAmount(Integer prizeAmount) {
      this.prizeAmount = prizeAmount;
    }

    public Integer getPrizeAmountAdjusted() {
      return this.prizeAmountAdjusted;
    }

    public void setPrizeAmountAdjusted(Integer prizeAmountAdjusted) {
      this.prizeAmountAdjusted = prizeAmountAdjusted;
    }

    public String getDateAwarded() {
      return this.dateAwarded;
    }

    public void setDateAwarded(String dateAwarded) {
      this.dateAwarded = dateAwarded;
    }

    public List<? extends Links> getLinks() {
      return this.links;
    }

    public void setLinks(List<? extends Links> links) {
      this.links = links;
    }

    public List<Laureates> getLaureates() {
      return this.laureates;
    }

    public void setLaureates(List<Laureates> laureates) {
      this.laureates = laureates;
    }

    public CategoryFullName getCategory() {
      return this.category;
    }

    public void setCategory(CategoryFullName category) {
      this.category = category;
    }

    public Laureates.Motivation getTopMotivation() {
      return this.topMotivation;
    }

    public void setTopMotivation(Laureates.Motivation topMotivation) {
      this.topMotivation = topMotivation;
    }

    public static class CategoryFullName implements Serializable {
      private String no;

      private String se;

      private String en;

      public String getNo() {
        return this.no;
      }

      public void setNo(String no) {
        this.no = no;
      }

      public String getSe() {
        return this.se;
      }

      public void setSe(String se) {
        this.se = se;
      }

      public String getEn() {
        return this.en;
      }

      public void setEn(String en) {
        this.en = en;
      }
    }

    public static class Links implements Serializable {
      private String types;

      private String rel;

      private String action;

      private String href;

      public String getTypes() {
        return this.types;
      }

      public void setTypes(String types) {
        this.types = types;
      }

      public String getRel() {
        return this.rel;
      }

      public void setRel(String rel) {
        this.rel = rel;
      }

      public String getAction() {
        return this.action;
      }

      public void setAction(String action) {
        this.action = action;
      }

      public String getHref() {
        return this.href;
      }

      public void setHref(String href) {
        this.href = href;
      }
    }

    public static class Laureates implements Serializable {
      private String portion;

      private String sortOrder;

      private Motivation motivation;

      private Motivation fullName;

      private List<? extends Links> links;

      private String id;

      private Motivation knownName;

      public String getPortion() {
        return this.portion;
      }

      public void setPortion(String portion) {
        this.portion = portion;
      }

      public String getSortOrder() {
        return this.sortOrder;
      }

      public void setSortOrder(String sortOrder) {
        this.sortOrder = sortOrder;
      }

      public Motivation getMotivation() {
        return this.motivation;
      }

      public void setMotivation(Motivation motivation) {
        this.motivation = motivation;
      }

      public Motivation getFullName() {
        return this.fullName;
      }

      public void setFullName(Motivation fullName) {
        this.fullName = fullName;
      }

      public List<? extends Links> getLinks() {
        return this.links;
      }

      public void setLinks(List<? extends Links> links) {
        this.links = links;
      }

      public String getId() {
        return this.id;
      }

      public void setId(String id) {
        this.id = id;
      }

      public Motivation getKnownName() {
        return this.knownName;
      }

      public void setKnownName(Motivation knownName) {
        this.knownName = knownName;
      }

      public static class Motivation implements Serializable {
        private String en;

        public String getEn() {
          return this.en;
        }

        public void setEn(String en) {
          this.en = en;
        }
      }
    }
  }
}


Główną klasą jest `NobelPrizeResponse`, zawierającą metadane oraz listę nagród nobla - w tej aplikacji nie wykorzystamy wszystkich danych, lecz możemy pozostawić wygenerowane klasy bez zmian.

Do pakietu `api` dodajmy interfejs `NobelPrizeApi` i zdefiniujmy metodę zwracającą listę nagród dla zadanej kategorii. Będziemy zapytania wykonywać **asynchronicznie**.

In [ ]:
public interface NobelPrizeApi {
    @GET("2.1/nobelPrizes")
    Call<NobelAwardsResponse> getNobelPrizes(
            @Query("limit") int limit,
            @Query("sort") String sort,
            @Query("nobelPrizeYear") int yearFrom,
            @Query("yearTo") int yearTo,
            @Query("nobelPrizeCategory") String category
    );
}

Następnie utwórzmy instancję `Retrofit` - do pakietu `api` dodajmy obiekt `RetrofitInstance`. Wykorzystamy również `HttpLoggingInterceptor` w celu sprawdzenia odpowiedzi.

In [ ]:
public class RetrofitInstance {
    private RetrofitInstance(){}

    private static volatile NobelPrizeApi api;
    private static final String baseUrl = "https://api.nobelprize.org/";

    public static NobelPrizeApi getApi() {
        if (api == null) {
            synchronized (RetrofitInstance.class) {
                if (api == null) {
                    HttpLoggingInterceptor interceptor = new HttpLoggingInterceptor();
                    interceptor.setLevel(HttpLoggingInterceptor.Level.BODY);
                    OkHttpClient client = new OkHttpClient.Builder()
                            .addInterceptor(interceptor)
                            .build();
                    api = new Retrofit.Builder()
                            .baseUrl(baseUrl)
                            .addConverterFactory(GsonConverterFactory.create())
                            .client(client)
                            .build().create(NobelPrizeApi.class);
                }
            }
        }
        return api;
    }
}

Do manifestu dodajmy upoważnienie na dostęp do internetu

In [ ]:
<uses-permission android:name="android.permission.INTERNET"/>

### **szkielet `MVVM`**

Rozpocznijmy od repozytorium (w pakiecie `repository`) - zawiera jedną metodę `getNobelPrizes`

In [ ]:
public class NobelRepository {
    public Call<NobelAwardsResponse> getNobelPrizes (
            int limit,
            String sort,
            int yearFrom,
            int yearTo,
            String category){
        return RetrofitInstance.getApi().getNobelPrizes(
                limit, sort, yearFrom, yearTo, category);
    }
}

Dodajmy do pakietu zawierającego `NobelAwardsFragment` (`ui.fragments.nobelawards`) odpowiedni `ViewModel`

In [ ]:
public class NobelAwardsViewModel extends ViewModel {
    private final NobelRepository repository = new NobelRepository();
    private final MutableLiveData<NobelAwardsResponse> nobelPrizes = new MutableLiveData<>();

    public LiveData<NobelAwardsResponse> getNobelPrizes() {
        return nobelPrizes;
    }
}

Odbieramy dane jako `NobelPrizeResponse` - przechowujemy je jako `MutableLiveData` w prywatnej wartości `nobelPrizes`. Dodajemy getter - zwraca `LiveData` (niemutowalne).

Następnie zdefiniujmy funkcję zwracającą listę wszystkich nagród dla zadanej kategorii.

In [ ]:
    public void getNobelPrizes(
            int limit,
            String sort,
            int yearFrom,
            int yearTo,
            String category){}

W pierwszym kroku, zdefiniujmy obiekt `Call`, który odbierzemy

In [ ]:
Call<NobelAwardsResponse> call = repository.getNobelPrizes(
        limit, sort, yearFrom, yearTo, category);

Wykorzystamy metofdę `enqueue` do odebrania danych

In [ ]:
public void getNobelPrizes(
        int limit,
        String sort,
        int yearFrom,
        int yearTo,
        String category){
        Call<NobelAwardsResponse> call = repository.getNobelPrizes(
                limit, sort, yearFrom, yearTo, category);

        call.enqueue(new Callback<NobelAwardsResponse>() {
            @Override
            public void onResponse(@NonNull Call<NobelAwardsResponse> call, 
                                   @NonNull Response<NobelAwardsResponse> response) {
                if (response.isSuccessful()){
                    NobelAwardsResponse nobelAwards = response.body();
                    if (nobelAwards != null)
                        nobelPrizes.postValue(nobelAwards);
                }
            }

            @Override
            public void onFailure(@NonNull Call<NobelAwardsResponse> call, 
                                  @NonNull Throwable t) {
                Log.e(TAG, "error: " + t.getMessage() + "at " + TAG);
            }
        });
}

W metodzie `onResponse` dostajemy obiekt `Response` w którym znajdują się nasze dane. Po sprawdzeniu czy obiekt nie jest `null` wykonujemy metodę `postValue` na naszym `MutableLiveData`.

Ponieważ nie przy starcie aplikacji chcemy wyświetlić dane - tutaj będą to wszystkie nagrody dla fizyki - musimy określić kategorie. Przejdźmy do pliku `Constants` w pakiecie `util` i dodajmy klasę wyliczeniową wraz z mapą pozwalającą na wybór kategorii.

In [ ]:
public final class Constants {
    private Constants(){}

    public enum CATEGORIES{
        PHYSICS, CHEMISTRY, LITERATURE, PEACE, ECONOMY, PHYSIOLOGYORMEDICINE
    }

    public static Map<CATEGORIES, String> categories = new TreeMap<CATEGORIES, String>(){{
        put(CATEGORIES.PHYSICS, "phy");
        put(CATEGORIES.ECONOMY, "eco");
        put(CATEGORIES.LITERATURE, "lit");
        put(CATEGORIES.PEACE, "pea");
        put(CATEGORIES.CHEMISTRY, "che");
        put(CATEGORIES.PHYSIOLOGYORMEDICINE, "med");
    }};
}

W naszym `NobelAwardsViewModel` wywołajmy funkcję `getNobelPrizes` w konstruktorze - co pozwoli nam zainicjować listę przy starcie aplikacji.

In [ ]:
public NobelAwardsViewModel(){
    getNobelPrizes(200, "desc", 1901, 2022, "phy");
}

### **`NobelAwardsFragment`**

Przejdźmy do fragmentu i rozpocznijmy od layoutu

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <Spinner
        android:id="@+id/categorySpinner"
        style="@style/Widget.AppCompat.Spinner"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:drawSelectorOnTop="true"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/nobelPrizeRV"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_margin="8dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/categorySpinner" />

    <ProgressBar
        android:id="@+id/nobelPrizeProgressBar"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:layout_marginTop="8dp"
        android:background="@android:color/transparent"
        android:visibility="invisible"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/categorySpinner" />

</androidx.constraintlayout.widget.ConstraintLayout>

Wykorzystamy trzy elementy
- `Spinner` do wyboru kategorii
- `RecyclerView` do wyświetlenia listy
- `ProgressBar` wyświetlany przy ładowaniu danych

Zdefiniujmy layout dla elementu spinnera

In [ ]:
<TextView xmlns:android="http://schemas.android.com/apk/res/android"
    android:id="@+id/spinner_layout"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:ellipsize="marquee"
    android:fontFamily="sans-serif"
    android:gravity="center"
    android:singleLine="true"
    android:text=""
    android:padding="10dp"
    android:textSize="24sp" />

oraz dla pojedynczego elementu `RecyclerView`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>

<androidx.cardview.widget.CardView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    app:cardCornerRadius="30dp"
    app:cardElevation="15dp"
    app:cardBackgroundColor="@color/teal_200"
    android:layout_margin="8dp"
    >
<androidx.constraintlayout.widget.ConstraintLayout
    android:layout_width="match_parent"
    android:layout_height="wrap_content"

    android:layout_marginTop="16dp"
    >

        <TextView
            android:id="@+id/year"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_margin="8dp"
            android:layout_marginBottom="46dp"
            android:text="2000"
            android:textSize="24sp"
            app:layout_constraintBottom_toBottomOf="@+id/motivation"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="parent" />

        <TextView
            android:id="@+id/listOfLaureates"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_margin="8dp"
            android:fontFamily="sans-serif-smallcaps"
            android:text="Syukuro Manabe, Klaus Hasselmann, Klaus Hasselmann, Klaus Hasselmann"
            android:textAlignment="textStart"
            android:textSize="18sp"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toEndOf="@+id/year"
            app:layout_constraintTop_toTopOf="parent" />

        <TextView
            android:id="@+id/motivation"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_margin="8dp"
            android:fontFamily="sans-serif-black"
            android:text="for groundbreaking contributions to our understanding of complex physical systems"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toEndOf="@+id/year"
            app:layout_constraintTop_toBottomOf="@+id/listOfLaureates" />

</androidx.constraintlayout.widget.ConstraintLayout>

</androidx.cardview.widget.CardView>

Kolejnym niezbędnym elementem jest adapter dla `RecyclerView` - dodajmy pakiet `adapters.nobelprizes` - skorzystamy z `ListAdapter`, więc musimy utworzyć trzy klasy. Rozpocznijmy od `ViewHolder`

In [ ]:
public class NobelPrizeViewHolder extends RecyclerView.ViewHolder {
    private final NobelPrizeRvItemBinding binding;
    public NobelPrizeViewHolder(NobelPrizeRvItemBinding binding) {
        super(binding.getRoot());
        this.binding = binding;
    }

Ponieważ dostęp do laureatów nagrody mamy przez listę, a chcemy wyświetlić ich `FullName` w polu `TextView`, dodajmy metodę zwracającą sformatowany `String`

In [ ]:
private String laureates(NobelAwardsResponse.NobelPrizes item){
    StringBuilder laureates = new StringBuilder();
    if (item.getLaureates() != null)
        for(NobelAwardsResponse.NobelPrizes.Laureates laureate : item.getLaureates()) {
            if (laureate.getFullName().getEn() != null && laureate.getFullName() != null)
                laureates.append(laureate.getFullName().getEn()).append(" ");
            else laureates.append("no name");
        }
    else
        laureates.append("no names");
    return laureates.toString();
}

Tutaj musimy zwrócić uwagę, że pola które otrzymujemy z serwera mogą być puste (`null`), musimy te przypadki również obsłużyć. Dodajmy funkcję `bind`

In [ ]:
public void bind(NobelAwardsResponse.NobelPrizes item){
    binding.year.setText(item.getAwardYear());
    if (item.getTopMotivation() != null && item.getTopMotivation().getEn() != null)
        binding.motivation.setText(item.getTopMotivation().getEn());
    else
        binding.motivation.setText("not specified");

    binding.listOfLaureates.setText(laureates(item));
}

Zdefiniujmy `Comparator` oraz `Adapter`

In [ ]:
public class NobelPrizeComparator extends 
    DiffUtil.ItemCallback<NobelAwardsResponse.NobelPrizes> {
    @Override
    public boolean areItemsTheSame(@NonNull NobelAwardsResponse.NobelPrizes oldItem, 
                                   @NonNull NobelAwardsResponse.NobelPrizes newItem) {
        return newItem == oldItem;
    }

    @Override
    public boolean areContentsTheSame(@NonNull NobelAwardsResponse.NobelPrizes oldItem, 
                                      @NonNull NobelAwardsResponse.NobelPrizes newItem) {
        return newItem.getAwardYear().equals(oldItem.getAwardYear());
    }
}


In [ ]:
public class NobelPrizeAdapter extends 
    ListAdapter<NobelAwardsResponse.NobelPrizes, NobelPrizeViewHolder> {
    public NobelPrizeAdapter(NobelPrizeComparator itemComparator) {
        super(itemComparator);
    }

    @NonNull
    @Override
    public NobelPrizeViewHolder onCreateViewHolder(@NonNull ViewGroup parent, 
                                                   int viewType) {
        return new NobelPrizeViewHolder(NobelPrizeRvItemBinding.inflate(
                LayoutInflater.from(parent.getContext()), parent, false
        ));
    }

    @Override
    public void onBindViewHolder(@NonNull NobelPrizeViewHolder holder, int position) {
        NobelAwardsResponse.NobelPrizes item = getItem(position);
        holder.bind(item);
    }
}


Przejdźmy do `NobelAwardsFragment` - dodajmy `viewModel`

In [ ]:
private NobelAwardsViewModel viewModel;

...
    
    @Override
    public void onViewCreated(@NonNull View view, 
                              @Nullable Bundle savedInstanceState) {
        super.onViewCreated(view, savedInstanceState);

        viewModel = new ViewModelProvider(this).get(NobelAwardsViewModel.class);
        ...
    }
...

Nastrępnie zdefiniujmy funkcję pomocniczą dla `RecyclerView`

In [ ]:
private void setupRecyclerView(NobelPrizeAdapter adapter){
    binding.nobelPrizeRV
        .setAdapter(adapter);
    binding.nobelPrizeRV
        .setLayoutManager(new LinearLayoutManager(requireContext()));
}

Następnie oddajmy funkcję pomocniczą dla `Spinner`

In [ ]:
private void setupSpinner(){
    binding.categorySpinner.setAdapter(
            new ArrayAdapter<>(requireContext(),
                    R.layout.spinner_nobel_award_layout,
                    (Constants.categories.values().toArray())));
    binding.categorySpinner
        .setOnItemSelectedListener(new AdapterView.OnItemSelectedListener() {
        @Override
        public void onItemSelected(AdapterView<?> parent, 
                                   View view, 
                                   int position, 
                                   long id) {
            viewModel.getNobelPrizes(200, "desc", 1901, 2022,
                    (Constants.categories.values().toArray()[position].toString()));
        }

        @Override
        public void onNothingSelected(AdapterView<?> parent) {

        }
    });
}

Finalnie, zdefiniujmy w metodzie `onViewCreated` zdefiniujmy obserwator

In [ ]:
@Override
public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
    super.onViewCreated(view, savedInstanceState);

    viewModel = new ViewModelProvider(this).get(NobelAwardsViewModel.class);

    NobelPrizeAdapter adapter = new NobelPrizeAdapter(new NobelPrizeComparator());
    setupRecyclerView(adapter);

    viewModel.getNobelPrizes().observe(getViewLifecycleOwner(), nobelAwardsResponse -> {
        adapter.submitList(nobelAwardsResponse.getNobelPrizes());
    });

    setupSpinner();
}

Możemy przetestować aplikację

<img src="https://media0.giphy.com/media/NuuG5HnAdmdTjLV0DV/giphy.webp" width="150" />

### **`NobelAwardFragment`**

Obsłużymy teraz kolejny fragment - wyświetlający pełniejszą informację o zadanej nagrodzie. Musimy jednoznacznie określić nagrodę - ponieważ nie mamy dostępnego identyfikatora, posłużymy się rokiem oraz kategorią.

Z fragmentu `NobelAwardsFragment` prześlemy dwie informacje (`awardYear`, `category`) do `NobelAwardFragment`, następnie połączymy się z serwerem i wykonamy odpowiednie żądanie. Istnieje również inny sposób na osiągnięcie tego samoego rezultatu - ponieważ mamy już dostępne wszystkie dane, możemy również przekazać cały obiekt (tutaj należy wwykorzystać interfejs `Serializable`, lub `Parcelable`).

Po pierwsze dodajmy odpowiednie argumenty do `navigation`.

In [ ]:
<fragment
    android:id="@+id/nobelAwardsFragment"
    android:name="pl.udu.uwr.pum.verynobleappjava.ui.fragments.nobleawards.NobelAwardsFragment"
    android:label="Nagrody Nobla"
    tools:layout="@layout/fragment_nobel_awards" >
    <action
        android:id="@+id/action_nobelAwardsFragment_to_nobelAwardFragment"
        app:destination="@id/nobelAwardFragment" >
        <argument
            android:name="category"
            app:argType="string" />
        <argument
            android:name="awardYear"
            app:argType="string" />
    </action>
</fragment>

Dodajmy `onClick` do funkcji `bind` klasy `NobelPrizeViewHolder`

In [ ]:
binding.getRoot().setOnClickListener(v -> {
    NavDirections action = NobleAwardsFragmentDirections
            .actionNobleAwardsFragmentFragmentToNobelAwardFragment(
                item.getCategory().getEn(), item.getAwardYear()
            );
    Navigation.findNavController(binding.getRoot()).navigate(action);
});

Dodajmy funkcję `getNobelPrize` do `NobelPrizeApi`, która przyjmować będzie kategorię i rok

In [ ]:
@GET("2.1/nobelPrizes")
Call<NobelAwardsResponse> getNobelPrize(
        @Query("nobelPrizeYear") int year,
        @Query("nobelPrizeCategory") String category
);

Dodajmy również odpowiednią metodę do `NobelRepository`

In [ ]:
public Call<NobelAwardsResponse> getNobelPrize (
        int year, String category
){
    return RetrofitInstance.getApi().getNobelPrize(year, category);
}

Przygotujmy layout dla `NobelAwardFragment`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".ui.fragments.nobelaward.NobelAwardFragment">

    <ProgressBar
        android:id="@+id/nobelPrizeProgressBar"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:layout_marginTop="8dp"
        android:background="@android:color/transparent"
        android:visibility="invisible"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/dateAwardTextView" />

    <TextView
        android:id="@+id/categoryFullNameTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginTop="16dp"
        android:fontFamily="sans-serif-smallcaps"
        android:text="Physics"
        android:textSize="24sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/dateAwardTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:fontFamily="sans-serif"
        android:text="Date awarded"
        android:textSize="24sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/categoryFullNameTextView" />

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/nobelAwardRV"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_margin="8dp"
        app:layout_constraintTop_toBottomOf="@+id/dateAwardTextView" />

</androidx.constraintlayout.widget.ConstraintLayout>

Również tutaj będziemy mieli `RecyclerView` - wyświetlimy w nim listę laureatów nagrody. Dodajmy layout dla pojedynczego elementu.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>

<androidx.cardview.widget.CardView 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    app:cardCornerRadius="30dp"
    app:cardElevation="15dp"
    app:cardBackgroundColor="@color/teal_200"
    android:layout_margin="8dp"
    >

<androidx.constraintlayout.widget.ConstraintLayout
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_margin="16dp">


    <TextView
        android:id="@+id/fullNameTextView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_margin="8dp"
        android:fontFamily="sans-serif-smallcaps"
        android:text="Full Name"
        android:textSize="24sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/textView2"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_margin="8dp"
        android:text="Portion: "
        android:textSize="20sp"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/fullNameTextView" />

    <TextView
        android:id="@+id/portionTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_margin="8dp"
        android:text="1/4"
        android:textSize="20sp"
        app:layout_constraintStart_toEndOf="@+id/textView2"
        app:layout_constraintTop_toBottomOf="@+id/fullNameTextView" />

    <TextView
        android:id="@+id/motivationTextView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_margin="8dp"
        android:text="motivation"
        android:textSize="16sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/portionTextView" />

</androidx.constraintlayout.widget.ConstraintLayout>
</androidx.cardview.widget.CardView>

Skoro mamy `RecyclerView`, musimy zaimplementować odpowiedni adapter - do pakietu `adapters` dodajmy pakiet `nobelprizelaureates` i tworzymy klasy `LaureateViewHolder`, `LaureateComparator`, `LaureateAdapter`

In [ ]:
class LaureateViewHolder(private val binding: LaureateRvItemBinding)
    : RecyclerView.ViewHolder(binding.root){
        fun bind(item: Laureate){
            binding.fullNameTextView.text = item.fullName?.en?: "missing name"
            binding.portionTextView.text = item.portion
            binding.motivationTextView.text = item.motivation.en
        }
}

In [ ]:
public class LaureateComparator extends 
    DiffUtil.ItemCallback<NobelAwardsResponse.NobelPrizes.Laureates> {
    @Override
    public boolean areItemsTheSame(
        @NonNull NobelAwardsResponse.NobelPrizes.Laureates oldItem, 
        @NonNull NobelAwardsResponse.NobelPrizes.Laureates newItem) {
        return oldItem.getId().equals(newItem.getId());
    }

    @Override
    public boolean areContentsTheSame(
        @NonNull NobelAwardsResponse.NobelPrizes.Laureates oldItem,
        @NonNull NobelAwardsResponse.NobelPrizes.Laureates newItem) {
        return newItem.getId().equals(oldItem.getId());
    }
}

In [ ]:
public class LaureateAdapter extends 
    ListAdapter<NobelAwardsResponse.NobelPrizes.Laureates, LaureateViewHolder> {
    public LaureateAdapter(LaureateComparator itemComparator) {
        super(itemComparator);
    }

    @NonNull
    @Override
    public LaureateViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return new LaureateViewHolder(LaureateRvItemBinding.inflate(
                LayoutInflater.from(parent.getContext()), parent, false
        ));
    }

    @Override
    public void onBindViewHolder(@NonNull LaureateViewHolder holder, int position) {
        NobelAwardsResponse.NobelPrizes.Laureates item = getItem(position);
        holder.bind(item);
    }
}

Do pakieru `ui.fragments` dodajmy pakiet `nobelaward` i przenieśmy do niego klasę `NobelAwardFragment`. W tym pakiecie tworzymy również `NobelAwardViewModel`

In [ ]:
public class NobelAwardViewModel extends ViewModel {
    private final NobelRepository repository = new NobelRepository();
    private final MutableLiveData<NobelAwardsResponse> nobelPrize = new MutableLiveData<>();

    public void getNobelPrize(
            int year,
            String category){
        Call<NobelAwardsResponse> call = repository.getNobelPrize(year, category);

        call.enqueue(new Callback<NobelAwardsResponse>() {
            @Override
            public void onResponse(@NonNull Call<NobelAwardsResponse> call, 
                                   @NonNull Response<NobelAwardsResponse> response) {
                if (response.isSuccessful()){
                    NobelAwardsResponse nobelAwards = response.body();
                    if (nobelAwards != null)
                        nobelPrize.postValue(nobelAwards);
                }
            }

            @Override
            public void onFailure(@NonNull Call<NobelAwardsResponse> call, 
                                  @NonNull Throwable t) {
                Log.e(TAG, "error: " + t.getMessage() + "at " + TAG);
            }
        });
    }

    public LiveData<NobelAwardsResponse> getNobelPrize() {
        return nobelPrize;
    }
}

W `NobelAwardFragment` tworzymy `viewmodel` oraz odbieramy przesłane argumenty.

In [ ]:
public class NobelAwardFragment extends Fragment {

    private FragmentNobelAwardBinding binding;

    private String category;
    private String awardYear;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        binding = FragmentNobelAwardBinding.inflate(inflater, container, false);
        category = requireArguments().getString("category");
        awardYear = requireArguments().getString("awardYear");
        return binding.getRoot();
    }
    ...
}

Podobnie jak w poprzednim fragmencie dodajmy funkcję pomocniczą

In [ ]:
private void setupRecyclerView(LaureateAdapter adapter){
    binding.nobelAwardRV.setAdapter(adapter);
    binding.nobelAwardRV.setLayoutManager(new LinearLayoutManager(requireContext()));
}

W przypadku poprawnej odpowiedzi ustawiamy odpowiednie pola, oraz przesyłamy listę laureatów do adaptera `RecyclerView`.

Zwróćmy uwagę na to że `nobelPrizes` jest listą

In [ ]:
data class NobelPrizeResponse(
    val meta: Meta,
    val nobelPrizes: List<NobelPrize>
)

Ale wiemy że w tym przypadku dostaniemy zawsze jeden element w liście, więc posługujemy się funkcją `findFirst` aby uzyskać do niego dostęp

In [ ]:
@Override
public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
    super.onViewCreated(view, savedInstanceState);
    NobelAwardViewModel viewModel = new ViewModelProvider(this).get(NobelAwardViewModel.class);

    viewModel.getNobelPrize(
        Integer.parseInt(awardYear), 
        Constants.categories.get(
            Constants
            .CATEGORIES
            .valueOf(category.toUpperCase().replaceAll("\\s+",""))));

    LaureateAdapter adapter = new LaureateAdapter(new LaureateComparator());

    setupRecyclerView(adapter);

    viewModel.getNobelPrize().observe(getViewLifecycleOwner(), nobelAwardsResponse -> {
        NobelAwardsResponse.NobelPrizes item = 
            nobelAwardsResponse.getNobelPrizes().stream().findFirst().orElse(null);
        if (item != null) {
            if (item.getCategoryFullName() != null && 
                item.getCategoryFullName().getEn() != null)
                binding.categoryFullNameTextView.setText(item.getCategoryFullName().getEn());
            binding.dateAwardTextView.setText(item.getDateAwarded());
            adapter.submitList(item.getLaureates());
        }
    });

}

Możemy przetestować aplikację

<img src="https://media3.giphy.com/media/6pIR2WozCb81CcuGZo/giphy.webp" width="150" />

### **`LaureateFragment`**

Fragment będzie wyświetlał informacje o wybranym laureacie. W `NobelAwardFragment` mamy listę laureatów jednej, konkretnej nagrody nobla. Dodamy obsługę `onClick` i przekażemy `id` laureata do `LaureateFragment`, który wyświetli pełniejsze informacje. Tym razem skorzystamy z innego endpointa **/laureate/{laureateID}**. Przykładowe żądanie: http://api.nobelprize.org/2.1/laureate/11

Tutaj mamy tylko jeden parametr
```xml
- name: `laureateID`  
  in: path  
  required: `true`  
  description: ID laureata.  
  type: `integer`  
  minimum: 1
```

Odpowiedź wygląda następująco

In [ ]:
[
  {
    "id": "11",
    "knownName": {
      "en": "Albert A. Michelson",
      "se": "Albert A. Michelson"
    },
    "givenName": {
      "en": "Albert A.",
      "se": "Albert A."
    },
    "familyName": {
      "en": "Michelson",
      "se": "Michelson"
    },
    "fullName": {
      "en": "Albert Abraham Michelson",
      "se": "Albert Abraham Michelson"
    },
    "fileName": "michelson",
    "gender": "male",
    "birth": {
      "date": "1852-12-19",
      "place": {
        "city": {
          "en": "Strelno",
          "no": "Strelno",
          "se": "Strelno"
        },
        "country": {
          "en": "Prussia",
          "no": "Preussen",
          "se": "Preussen"
        },
        "cityNow": {
          "en": "Strzelno",
          "no": "Strzelno",
          "se": "Strzelno",
          "sameAs": [
            "https://www.wikidata.org/wiki/Q1005414",
            "https://www.wikipedia.org/wiki/Strzelno"
          ]
        },
        "countryNow": {
          "en": "Poland",
          "no": "Polen",
          "se": "Polen",
          "sameAs": [
            "https://www.wikidata.org/wiki/Q36"
          ]
        },
        "continent": {
          "en": "Europe",
          "no": "Europa",
          "se": "Europa"
        },
        "locationString": {
          "en": "Strelno, Prussia (now Strzelno, Poland)",
          "no": "Strelno, Preussen (nå Strzelno, Polen)",
          "se": "Strelno, Preussen (nu Strzelno, Polen)"
        }
      }
    },
    "death": {
      "date": "1931-05-09",
      "place": {
        "city": {
          "en": "Pasadena, CA",
          "no": "Pasadena, CA",
          "se": "Pasadena, CA"
        },
        "country": {
          "en": "USA",
          "no": "USA",
          "se": "USA",
          "sameAs": "https://www.wikidata.org/wiki/Q30"
        },
        "cityNow": {
          "en": "Pasadena, CA",
          "no": "Pasadena, CA",
          "se": "Pasadena, CA",
          "sameAs": [
            "https://www.wikidata.org/wiki/Q485176",
            "https://www.wikipedia.org/wiki/Pasadena,_California"
          ]
        },
        "countryNow": {
          "en": "USA",
          "no": "USA",
          "se": "USA",
          "sameAs": [
            "https://www.wikidata.org/wiki/Q30"
          ]
        },
        "continent": {
          "en": "North America",
          "no": "Nord-Amerika",
          "se": "Nordamerika"
        },
        "locationString": {
          "en": "Pasadena, CA, USA",
          "no": "Pasadena, CA, USA",
          "se": "Pasadena, CA, USA"
        }
      }
    },
    "wikipedia": {
      "slug": "Albert_Abraham_Michelson",
      "english": "https://en.wikipedia.org/wiki/Albert_Abraham_Michelson"
    },
    "wikidata": {
      "id": "Q127234",
      "url": "https://www.wikidata.org/wiki/Q127234"
    },
    "sameAs": [
      "https://www.wikidata.org/wiki/Q127234",
      "https://en.wikipedia.org/wiki/Albert_Abraham_Michelson"
    ],
    "links": [
      {
        "rel": "laureate",
        "href": "https://api.nobelprize.org/2/laureate/11",
        "action": "GET",
        "types": "application/json"
      },
      {
        "rel": "external",
        "href": "https://www.nobelprize.org/laureate/11",
        "title": "Albert A. Michelson - Facts",
        "action": "GET",
        "types": "text/html",
        "class": [
          "laureate facts"
        ]
      }
    ],
    "nobelPrizes": [
      {
        "awardYear": "1907",
        "category": {
          "en": "Physics",
          "no": "Fysikk",
          "se": "Fysik"
        },
        "categoryFullName": {
          "en": "The Nobel Prize in Physics",
          "no": "Nobelprisen i fysikk",
          "se": "Nobelpriset i fysik"
        },
        "sortOrder": "1",
        "portion": "1",
        "prizeStatus": "received",
        "motivation": {
          "en": "for his optical precision instruments and the spectroscopic and metrological investigations carried out with their aid",
          "se": "för hans optiska precisionsinstrument och hans därmed utförda spektroskopiska och metrologiska undersökningar"
        },
        "prizeAmount": 138796,
        "prizeAmountAdjusted": 7161123,
        "affiliations": [
          {
            "name": {
              "en": "University of Chicago",
              "no": "University of Chicago",
              "se": "University of Chicago"
            },
            "nameNow": {
              "en": "University of Chicago"
            },
            "city": {
              "en": "Chicago, IL",
              "no": "Chicago, IL",
              "se": "Chicago, IL"
            },
            "country": {
              "en": "USA",
              "no": "USA",
              "se": "USA"
            },
            "cityNow": {
              "en": "Chicago, IL",
              "no": "Chicago, IL",
              "se": "Chicago, IL",
              "sameAs": [
                "https://www.wikidata.org/wiki/Q1297",
                "https://www.wikipedia.org/wiki/Chicago"
              ]
            },
            "countryNow": {
              "en": "USA",
              "no": "USA",
              "se": "USA",
              "sameAs": [
                "https://www.wikidata.org/wiki/Q30"
              ]
            },
            "locationString": {
              "en": "Chicago, IL, USA",
              "no": "Chicago, IL, USA",
              "se": "Chicago, IL, USA"
            }
          }
        ],
        "links": [
          {
            "rel": "nobelPrize",
            "href": "https://api.nobelprize.org/2/nobelPrize/phy/1907",
            "action": "GET",
            "types": "application/json"
          },
          {
            "rel": "external",
            "href": "https://www.nobelprize.org/prizes/physics/1907/michelson/facts/",
            "title": "Albert A. Michelson - Facts",
            "action": "GET",
            "types": "text/html",
            "class": [
              "laureate facts"
            ]
          },
          {
            "rel": "external",
            "href": "https://www.nobelprize.org/prizes/physics/1907/summary/",
            "title": "The Nobel Prize in Physics 1907",
            "action": "GET",
            "types": "text/html",
            "class": [
              "prize summary"
            ]
          }
        ]
      }
    ],
    "meta": {
      "terms": "https://www.nobelprize.org/about/terms-of-use-for-api-nobelprize-org-and-data-nobelprize-org/",
      "license": "https://www.nobelprize.org/about/terms-of-use-for-api-nobelprize-org-and-data-nobelprize-org/#licence",
      "disclaimer": "https://www.nobelprize.org/about/terms-of-use-for-api-nobelprize-org-and-data-nobelprize-org/#disclaimer"
    }
  }
]

Tym razem wykorzystamy stronę https://json2csharp.com/code-converters/json-to-pojo do wygenerowania klas na podstawie Json.

In [ ]:
public class LaureateResponse extends ArrayList<LaureateResponseItem>{}

public class LaureateResponseItem {
    public String id;
    public KnownName knownName;
    public GivenName givenName;
    public FamilyName familyName;
    public FullName fullName;
    public String fileName;
    public String gender;
    public Birth birth;
    public Death death;
    public Wikipedia wikipedia;
    public Wikidata wikidata;
    public ArrayList<String> sameAs;
    public ArrayList<Link> links;
    public ArrayList<NobelPrize> nobelPrizes;
    public Meta meta;
}

public class Birth {
    public String date;
    public Place place;
}

public class Death {
    public String date;
    public Place place;
}

public class FamilyName {
    public String en;
    public String se;
}

public class FullName {
    public String en;
    public String se;
}

public class GivenName {
    public String en;
    public String se;
}

public class KnownName {
    public String en;
    public String se;
}

public class Link {
    public String rel;
    public String href;
    public String action;
    public String types;
    public String title;
}

public class Place {
    public City city;
    public Country country;
    public CityNow cityNow;
    public CountryNow countryNow;
    public Continent continent;
    public LocationString locationString;
}

public class Meta {
    public String terms;
    public String license;
    public String disclaimer;
}

public class Category {
    public String en;
    public String no;
    public String se;
}

public class CategoryFullName {
    public String en;
    public String no;
    public String se;
}

public class Country {
    public String en;
    public String no;
    public String se;
    public String sameAs;
}

public class CountryNow {
    public String en;
    public String no;
    public String se;
    public ArrayList<String> sameAs;
}

public class LocationString {
    public String en;
    public String no;
    public String se;
}

public class Motivation {
    public String en;
    public String se;
}

public class Name {
    public String en;
    public String no;
    public String se;
}

public class NameNow {
    public String en;
}

public class NobelPrize {
    public String awardYear;
    public Category category;
    public CategoryFullName categoryFullName;
    public String sortOrder;
    public String portion;
    public String dateAwarded;
    public String prizeStatus;
    public Motivation motivation;
    public int prizeAmount;
    public int prizeAmountAdjusted;
    public ArrayList<Affiliation> affiliations;
    public ArrayList<Link> links;
}

public class Affiliation {
    public Name name;
    public NameNow nameNow;
    public City city;
    public Country country;
    public CityNow cityNow;
    public CountryNow countryNow;
    public LocationString locationString;
}

public class City {
    public String en;
    public String no;
    public String se;
}

public class CityNow {
    public String en;
    public String no;
    public String se;
    public ArrayList<String> sameAs;
}

public class Continent {
    public String en;
    public String no;
    public String se;
}

Jak widać dostaliśmy koło 40 klas - jak wspominałem na początku nie będziemy specjalnie się przejmować tym co się dzieje dokładnie w tych klasach, więc zostawmy je tak jak zostały wygenerowane.

Zwróćmy uwagę na dwie pierwsze klasy
- `public class LaureateResponse extends ArrayList<LaureateResponseItem>` - odpowiedź dostajemy jako listę
- `LaureateResponseItem` - element listy

Dodajmy argument do `navigation`

In [ ]:
<fragment
    android:id="@+id/nobelAwardFragment"
    android:name="pl.udu.uwr.pum.verynobleappkotlin.ui.fragments.nobelaward.NobelAwardFragment"
    android:label="Nagroda Nobla"
    tools:layout="@layout/fragment_nobel_award" >
    <action
        android:id="@+id/action_nobelAwardFragment_to_nobelAwardsFragment"
        app:destination="@id/nobelAwardsFragment" />
    <action
        android:id="@+id/action_nobelAwardFragment_to_laureateFragment"
        app:destination="@id/laureateFragment" >
        <argument
            android:name="id"
            app:argType="string" />
    </action>
</fragment>

Dodajmy również `onClick` do funkcji `bind` klasy `LaureateViewHolder`

In [ ]:
binding.getRoot().setOnClickListener(v -> {
    NavDirections action = NobelAwardFragmentDirections
            .actionNobelAwardFragmentToLaureateFragment(
                    item.getId()
            );
    Navigation.findNavController(binding.getRoot()).navigate(action);
});

Przygotujmy layout `LaureateFragment`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:layout_margin="8dp">

    <ProgressBar
        android:id="@+id/laureateProgressBar"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:layout_marginTop="8dp"
        android:background="@android:color/transparent"
        android:visibility="invisible"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/fullNameTextView" />

    <TextView
        android:id="@+id/fullNameTextView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:fontFamily="sans-serif-smallcaps"
        android:gravity="center"
        android:text="Full Name"
        android:textSize="24sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/birth"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Birth:"
        android:textSize="20sp"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/fullNameTextView" />

    <TextView
        android:id="@+id/birthDateTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="8dp"
        android:text="19.01.2000"
        android:textSize="20sp"
        app:layout_constraintStart_toEndOf="@+id/birth"
        app:layout_constraintTop_toBottomOf="@+id/fullNameTextView" />

    <TextView
        android:id="@+id/death"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginEnd="8dp"
        android:text="Death:"
        android:textSize="20sp"
        app:layout_constraintEnd_toStartOf="@+id/deathDateTextView"
        app:layout_constraintTop_toBottomOf="@+id/fullNameTextView" />

    <TextView
        android:id="@+id/deathDateTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="19.01.2000"
        android:textSize="20sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/fullNameTextView" />

    <TextView
        android:id="@+id/birthCityTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="8dp"
        android:text="Warszawa"
        android:textSize="20sp"
        app:layout_constraintStart_toEndOf="@+id/birth"
        app:layout_constraintTop_toBottomOf="@+id/birthDateTextView" />

    <TextView
        android:id="@+id/deathCityTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginEnd="8dp"
        android:text="Warszawa"
        android:textSize="20sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/deathDateTextView" />

    <TextView
        android:id="@+id/birthCountryTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="8dp"
        android:text="Poland"
        android:textSize="20sp"
        app:layout_constraintStart_toEndOf="@+id/birth"
        app:layout_constraintTop_toBottomOf="@+id/birthCityTextView" />

    <TextView
        android:id="@+id/deathCountryTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Poland"
        android:textSize="20sp"
        android:layout_marginEnd="8dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/deathCityTextView" />

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/laureateRV"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/birthCountryTextView" />

    <Button
        android:id="@+id/wikiButton"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:text="wikipedia"
        android:layout_marginStart="36dp"
        android:layout_marginEnd="36dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/laureateRV" />

</androidx.constraintlayout.widget.ConstraintLayout>

Laureat może posiadać kilka nagród nobla, więc dodajmy również tutaj `RecyclerView` z listą wszystkich nagród otrzymanych przez laureata. W dostępnych danych mamy również link do strony na wikipedi, wykorzystamy to na kolejnym fragmencie, do którego przejdziemy dzięki dodanemu przyciskowi.

Dodajmy również layout pojedynczego elementu listy

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.cardview.widget.CardView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    app:cardCornerRadius="30dp"
    app:cardElevation="15dp"
    app:cardBackgroundColor="@color/teal_200"
    android:layout_margin="8dp"
    >

<androidx.constraintlayout.widget.ConstraintLayout
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_margin="16dp">

    <TextView
        android:id="@+id/yearTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="2000"
        android:textSize="20sp"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/categoryFullNameTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="4dp"
        android:layout_marginBottom="3dp"
        android:text="Physics"
        android:textSize="20sp"
        app:layout_constraintBottom_toBottomOf="@+id/yearTextView"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toEndOf="@+id/yearTextView"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/motivationTextView"
        android:layout_width="0dp"
        android:layout_height="wrap_content"
        android:layout_marginStart="8dp"
        android:lines="4"
        android:text="for groundbreaking contributions to our understanding of complex physical systems"
        android:textSize="12sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toEndOf="@+id/portion"
        app:layout_constraintTop_toBottomOf="@+id/affiliationsTextView" />

    <TextView
        android:id="@+id/portion"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="4dp"
        android:text="Portion:"
        android:textSize="8sp"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/affiliationsTextView" />

    <TextView
        android:id="@+id/portionTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="1/2"
        android:textSize="20sp"
        android:layout_marginStart="4dp"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/portion" />

    <TextView
        android:id="@+id/affiliationsTextView"
        android:layout_width="0dp"
        android:layout_height="wrap_content"
        android:gravity="center"
        android:text="Uniwersytet Wroclawski"
        android:textSize="20sp"
        app:layout_constraintEnd_toEndOf="parent"

        app:layout_constraintStart_toEndOf="@+id/affiliations"
        app:layout_constraintTop_toBottomOf="@+id/categoryFullNameTextView" />

    <TextView
        android:id="@+id/affiliations"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginBottom="4dp"
        android:text="Affiliations:"
        android:textSize="16sp"
        app:layout_constraintBottom_toTopOf="@+id/motivationTextView"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/yearTextView" />

</androidx.constraintlayout.widget.ConstraintLayout>
</androidx.cardview.widget.CardView>

Oraz przygotujmy adapter. Dodajmy pakiet `laureatenobelprize` do pakietu `adapters` i zaimplementujmy odpowiednie klasy

In [ ]:
public class NobelPrizeViewHolder extends RecyclerView.ViewHolder {
    private final LaureateNobelPrizeRvItemBinding binding;
    public NobelPrizeViewHolder(LaureateNobelPrizeRvItemBinding binding) {
        super(binding.getRoot());
        this.binding = binding;
    }

    public void bind(NobelPrize item){
        binding.categoryFullNameTextView.setText(item.categoryFullName.en);
        binding.portionTextView.setText(item.portion);
        binding.yearTextView.setText(item.awardYear);
        binding.motivationTextView.setText(item.motivation.en);
        binding.affiliationsTextView.setText(affiliations(item));
    }

    private String affiliations(NobelPrize item){
        StringBuilder affiliations = new StringBuilder();
        if (item.affiliations != null) {
            for (Affiliation affiliation : item.affiliations) {
                if (affiliation.name != null && affiliation.name.en != null)
                    affiliations.append(affiliation.name.en).append("\n");
            }
        } else
            affiliations.append("not specified");
        return affiliations.toString();
    }
}

Wyświetlamy nazwy instytutów z którymi jest powiązana dana osoba. Podobnie jak w przypadku `NobelPrizesViewHolder` dodajmy dodatkową metodę - mamy dostępną listę wszystkich instytutów, chcemy `String`.

In [ ]:
public class NobelPrizeComparator extends DiffUtil.ItemCallback<NobelPrize> {
    @Override
    public boolean areItemsTheSame(@NonNull NobelPrize oldItem, 
                                   @NonNull NobelPrize newItem) {
        return newItem == oldItem;
    }

    @Override
    public boolean areContentsTheSame(@NonNull NobelPrize oldItem, 
                                      @NonNull NobelPrize newItem) {
        return newItem.awardYear.equals(oldItem.awardYear);
    }
}


In [ ]:
public class NobelPrizeAdapter extends ListAdapter<NobelPrize, NobelPrizeViewHolder> {

    public NobelPrizeAdapter(NobelPrizeComparator itemComparator) {
        super(itemComparator);
    }

    @NonNull
    @Override
    public NobelPrizeViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return new NobelPrizeViewHolder(LaureateNobelPrizeRvItemBinding.inflate(
                LayoutInflater.from(parent.getContext()), parent, false
        ));
    }

    @Override
    public void onBindViewHolder(@NonNull NobelPrizeViewHolder holder, int position) {
        NobelPrize item = getItem(position);
        holder.bind(item);
    }
}

Do interfejsu `NobelPrizeApi` dodajmy nową funkcję

In [ ]:
@GET("2.1/laureate/{laureateID}")
Call<LaureateResponse> getLaureates(@Path("laureateID") String id);

dodajmy również `getLaureate` do repozytorium

In [ ]:
public Call<LaureateResponse> getLaureate (String id){
    return RetrofitInstance.getApi().getLaureates(id);
}

Do pakietu `ui.fragmenst.laureate` dodajmy `LaureateViewModel` - jest on analogiczny do poprzednich

In [ ]:
public class LaureateViewModel extends ViewModel {
    private final NobelRepository repository = new NobelRepository();
    private final MutableLiveData<LaureateResponse> laureate = new MutableLiveData<>();

    private final String TAG = "LaureateViewModel";

    public void getLaureate(String id){
        Call<LaureateResponse> call = repository.getLaureate(id);

        call.enqueue(new Callback<LaureateResponse>() {
            @Override
            public void onResponse(
                @NonNull Call<LaureateResponse> call, 
                @NonNull Response<LaureateResponse> response) {
                if (response.isSuccessful()){
                    LaureateResponse laureateResponse = response.body();
                    if (laureateResponse != null)
                        laureate.postValue(laureateResponse);
                }
            }

            @Override
            public void onFailure(
                @NonNull Call<LaureateResponse> call, 
                @NonNull Throwable t) {
                Log.e(TAG, "error: " + t.getMessage() + "at " + TAG);
            }
        });
    }

    public LiveData<LaureateResponse> getLaureate() {
        return laureate;
    }
}


W `LaureateFragment` odbieramy przesłany argument

In [ ]:
public class LaureateFragment extends Fragment {

    private FragmentLaureateBinding binding;

    private String id;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        binding = FragmentLaureateBinding.inflate(inflater, container, false);
        id = requireArguments().getString("id");
        return binding.getRoot();
    }
}

Podobnie jak w poprzednich fragmentach, dodajmy kilka funkcji

In [ ]:
private void setupRecyclerView(NobelPrizeAdapter adapter){
    binding.laureateRV.setAdapter(adapter);
    binding.laureateRV.setLayoutManager(new LinearLayoutManager(requireContext()));
}

Nasze dane otrzymujemy jako `ArrayList`, więc używamy odpowiednich metod dostępowych. Dodajemy również obsługę `onClick` naszego przycisku, dzięki któremu przechodzimy na `WikiFragment` - przekazujemy również `url` do wpisu w wikipedii.

In [ ]:
@Override
public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
    super.onViewCreated(view, savedInstanceState);
    LaureateViewModel viewModel = new ViewModelProvider(this).get(LaureateViewModel.class);

    viewModel.getLaureate(id);

    NobelPrizeAdapter adapter = new NobelPrizeAdapter(new NobelPrizeComparator());

    setupRecyclerView(adapter);

    viewModel.getLaureate().observe(getViewLifecycleOwner(), laureateResponse -> {
        binding.fullNameTextView.setText(laureateResponse.get(0).fullName.en);
        binding.birthDateTextView.setText(laureateResponse.get(0).birth.date);
        binding.birthCityTextView.setText(laureateResponse.get(0).birth.place.city.en);
        binding.birthCountryTextView.setText(laureateResponse.get(0).birth.place.country.en);
        if (laureateResponse.get(0).death != null) {
            binding.deathDateTextView.setText(laureateResponse.get(0).death.date);
            binding.deathCityTextView.setText(laureateResponse.get(0).death.place.city.en);
            binding.deathCountryTextView.setText(laureateResponse.get(0).death.place.country.en);
        }
        else {
            binding.deathDateTextView.setText("");
            binding.deathCityTextView.setText("");
            binding.deathCountryTextView.setText("");
        }
        adapter.submitList(laureateResponse.get(0).nobelPrizes);

        binding.wikiButton.setOnClickListener(v -> {
            NavDirections action = LaureateFragmentDirections
                    .actionLaureateFragmentToWikiLaureateFragmentFragment(
                            laureateResponse.get(0).wikipedia.english
                    );
            Navigation.findNavController(view).navigate(action);
        });
    });
}

### **`WikiFragment`**

W tym fragmencie wyświetlimy stronę internetową wiki - zrobimy to wykorzystując `WebView`. Rozpocznijmy od layoutu.

In [ ]:
<FrameLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".ui.fragments.wiki.WikiLaureateFragment">

    <WebView
        android:id="@+id/webView"
        android:layout_margin="8dp"
        android:layout_width="match_parent"
        android:layout_height="match_parent"/>

</FrameLayout>

W `WikiFragment` odbieramy argument

In [ ]:
public class WikiLaureateFragmentFragment extends Fragment {

    private FragmentWikiLaureateFragmentBinding binding;

    private String wikiUrl;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        binding = FragmentWikiLaureateFragmentBinding.inflate(inflater, container, false);
        wikiUrl = requireArguments().getString("url");
        return binding.getRoot();
    }
}

I w metodzie `onViewCreated` inicjujemy `WebView`

In [ ]:
@Override
public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
    super.onViewCreated(view, savedInstanceState);
    binding.webView.setWebViewClient(new WebViewClient());
    binding.webView.loadUrl(wikiUrl);
}

W pierwszym kroku tworzymy `WebViewClient` - jest to klasa która pozwala aplikacji przejąć kontrolę przy ładowaniu `url` do `WebView`. Jeżeli klient nie jest zainicjowany, `WebView` przekazuje kontrolę do `ActivityManager`, aby ten wybrał odpowiedni handler dla tego typu zasobu.

Następnie wykorzystujemy `loeadUrl` do załadowania adresu i wyświetlenia treści.

Możemy przetestować aplikację

<table><tr><td><img src="https://media0.giphy.com/media/NuuG5HnAdmdTjLV0DV/giphy.webp" width="150" /></td><td><img src="https://media3.giphy.com/media/6pIR2WozCb81CcuGZo/giphy.webp" width="150" /></td><td><img src="https://media0.giphy.com/media/8QoE7dYN8as4gRjE16/giphy.webp" width="150" /></td></tr></table>